# Murcielago System V1 - Training & testing

In [9]:
import torch
import torchaudio
from torch.utils.data import DataLoader
from custom_dataset import GunShotsNoisesDataset, split_dataset
import numpy as np
from cnn import ShotDetectionNetwork
from torch import nn
from train_test_functions import train_step, test_step, eval_model, accuracy_fn

## 1) Data preparing: Dataset and Dataloaders

In [10]:
metadata_file = "./dataset/metadata.xlsx"
audios_dir = "./dataset"
fs = 48000
scales = np.arange(1, 129)
transformation_dict = {"wavelet": "cmor","scales":scales}

GSN_visualize = GunShotsNoisesDataset(metadata_file, audios_dir, transformation_dict, fs, 0.01) #this returns audio waveform to analyse samples

GSN_dataset = GunShotsNoisesDataset(metadata_file, audios_dir, transformation_dict, fs)

In [11]:
batch_size = 4
train_data, test_data = split_dataset(GSN_dataset, 0.04)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [12]:
it = iter(train_dataloader)
first = next(it)
print(first[1])

tensor([1., 1., 1., 1.])


c:\Users\joaqo\anaconda3\envs\Sonido\Lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)


## 2) Model building

In [13]:
torch.manual_seed(42)
mur_cnn_v1 = ShotDetectionNetwork(64)
print(mur_cnn_v1)

ShotDetectionNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): Conv2d(64, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=1, bias=True)
  )
)


In [14]:
for p in mur_cnn_v1.parameters():
    print(p.grad)

None
None
None
None
None
None
None
None


In [15]:
for name, param in mur_cnn_v1.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

conv1.0.weight: requires_grad=True
conv1.0.bias: requires_grad=True
conv1.1.weight: requires_grad=True
conv1.1.bias: requires_grad=True
conv2.0.weight: requires_grad=True
conv2.0.bias: requires_grad=True
classifier.1.weight: requires_grad=True
classifier.1.bias: requires_grad=True


In [18]:
for name, module in mur_cnn_v1.named_modules():
        if isinstance(module, nn.Conv2d):
            print(module) 
        
        elif isinstance(module, nn.Linear):
            print(module)
        else:
            print("raro")

raro
raro
Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
Conv2d(64, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
raro
raro
Conv2d(16, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
raro
raro
raro
raro
LazyLinear(in_features=0, out_features=1, bias=True)


## 3) Train and Test

In [9]:
learn_rate = 0.01
loss_fn = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(mur_cnn_v1.parameters(), lr=learn_rate)

In [ ]:
epochs = 5

for epoch in range(epochs):
    print("-----------------------------------------------------------")
    print(f"Epoch: {epoch}")
    train_step(mur_cnn_v1, train_dataloader, loss_fn, optim, accuracy_fn=accuracy_fn)
    test_step(mur_cnn_v1, test_dataloader, loss_fn, accuracy_fn=accuracy_fn)
    

-----------------------------------------------------------
Epoch: 0
classifier.1 grad norm: 0.2734493613243103
classifier.1 grad norm: 0.0


c:\Users\joaqo\anaconda3\envs\Sonido\Lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)
c:\Users\joaqo\anaconda3\envs\Sonido\Lib\site-packages\torch\nn\modules\module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.2722097635269165
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.2709692716598511
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.2697279453277588
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.26848599314689636
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.2672435939311981
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.2660008668899536
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.26475805044174194
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.0
classifier.1 grad norm: 0.2635152339935303
classifier

In [10]:
for p in mur_cnn_v1.parameters():
    print(p.grad)

tensor([[[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]]])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([[

## 4) Results